In [287]:
'Test_Task1 seat or carrier seat Classification using XG Boost model and MLP model'

'Test_Task1 seat or carrier seat Classification using XG Boost model and MLP model'

In [288]:
import gc
gc.collect()

1495

In [289]:
%reset -f

In [290]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Data Preprocessing

# Test measurements

In [291]:
import os
import numpy as np
import pandas as pd

# Define path (2 levels up -> Data/Processed)
file_path = os.path.join("..", "..", "Data", "Test", "test_Emptyseat_npy_array_Lowpassfiltered_label.npy")

# Load the NumPy array from the file
loaded_array = np.load(file_path, mmap_mode='r')

# Convert to DataFrame
dataframe_testdata = pd.DataFrame(
    loaded_array,
    columns=['Frequency', 'FFT Magnitude', 'Phase', 'Object_Presence']
)

# Take first 2000 rows (or use sampling instead if needed)
dataframe_testdata = dataframe_testdata.iloc[:2000]
# dataframe_testdata = dataframe_testdata.sample(n=2000, random_state=42)

print(f"Data loaded from: {os.path.abspath(file_path)}")
dataframe_testdata


Data loaded from: /Users/mandarkale/Documents/MyProjects/MachineLearning/FIUS_Based_Infant_Presence_Detection_in_Car_Seats/source_code/Data/Test/test_Emptyseat_npy_array_Lowpassfiltered_label.npy


,Frequency,FFT Magnitude,Phase,Object_Presence
0,0.00000,17591.160470,0.000000,0.0
1,488.28125,8624.422770,-3.116442,0.0
2,976.56250,269.932200,2.781762,0.0
3,1464.84375,101.451438,1.259705,0.0
4,1953.12500,159.854316,-0.154286,0.0
...,...,...,...,...
1995,974121.09375,382.129825,0.512916,0.0
1996,974609.37500,385.904814,-2.873235,0.0
1997,975097.65625,363.592039,-0.043251,0.0
1998,975585.93750,330.245526,-2.947535,0.0


In [292]:
num_rows_test = len(dataframe_testdata)
num_rows_test

2000

In [293]:
fft_magnitudes_test = dataframe_testdata["FFT Magnitude"].values  # Convert to NumPy array
fft_magnitudes_test

array([17591.16046978,  8624.42277002,   269.93219957, ...,
         363.59203886,   330.24552634,   178.40396818], shape=(2000,))

In [294]:
fft_frequencies_test = dataframe_testdata["Frequency"].values  # Convert to NumPy array
fft_frequencies_test

array([0.00000000e+00, 4.88281250e+02, 9.76562500e+02, ...,
       9.75097656e+05, 9.75585937e+05, 9.76074219e+05], shape=(2000,))

In [295]:
fft_phase_test = dataframe_testdata["Phase"].values  # Convert to NumPy array
fft_phase_test

array([ 0.        , -3.11644234,  2.78176198, ..., -0.04325127,
       -2.94753519,  1.89234963], shape=(2000,))

In [296]:
import numpy as np
from scipy.stats import entropy
from scipy.signal import find_peaks

def extract_features(fft_freqs, fft_mags, fft_phase):
    # Normalize magnitudes to prevent division errors
    norm_mags = fft_mags / np.sum(fft_mags) if np.sum(fft_mags) > 0 else fft_mags
    
    # Spectral Centroid (Weighted Mean of Frequencies)
    spectral_centroid = np.sum(fft_freqs * norm_mags) / np.sum(norm_mags)

    # Spectral Bandwidth (Spread around centroid)
    spectral_bandwidth = np.sqrt(np.sum(norm_mags * (fft_freqs - spectral_centroid) ** 2))

    # Spectral Flatness (Geometric Mean / Arithmetic Mean)
    spectral_flatness = np.exp(np.mean(np.log(fft_mags + 1e-10))) / np.mean(fft_mags + 1e-10)

    # Phase Features
    phase_variance = np.var(fft_phase)  # Variance of phase angles
    phase_mean = np.mean(fft_phase)  # Mean of phase angles
    phase_diff = np.mean(np.diff(fft_phase))  # Average phase difference between frequencies

    return {
        "mean_fft": np.mean(fft_mags),
        "std_fft": np.std(fft_mags),
        "max_fft": np.max(fft_mags),
        "min_fft": np.min(fft_mags),
        "median_fft": np.median(fft_mags),
        "sum_fft": np.sum(fft_mags),
        "spectral_entropy": entropy(norm_mags),  # Energy spread
        "spectral_centroid": spectral_centroid,
        "spectral_bandwidth": spectral_bandwidth,
        "phase_variance": phase_variance,
        "phase_mean": phase_mean,
        "phase_diff": phase_diff,
    }


In [297]:
# Function to apply perturbation based on reference values
def add_perturbation(reference_features, num_rows, perturb_range=0.03):
    perturbed_data = []
    
    for _ in range(num_rows):
        perturbed_features = {
            key: value * (1 + np.random.uniform(-perturb_range, perturb_range)) 
            for key, value in reference_features.items()
        }
        perturbed_data.append(perturbed_features)
    
    return pd.DataFrame(perturbed_data)

In [298]:
# Extract features for fft_test
reference_features_test = extract_features(fft_frequencies_test, fft_magnitudes_test, fft_phase_test)
reference_features_test

{'mean_fft': np.float64(315.68738982606305),
 'std_fft': np.float64(463.08835097174983),
 'max_fft': np.float64(17591.160469784256),
 'min_fft': np.float64(7.302704927261968),
 'median_fft': np.float64(275.6937819378809),
 'sum_fft': np.float64(631374.7796521261),
 'spectral_entropy': np.float64(7.326219826670075),
 'spectral_centroid': np.float64(495862.10231327085),
 'spectral_bandwidth': np.float64(300149.40921247046),
 'phase_variance': np.float64(3.296296235765856),
 'phase_mean': np.float64(-0.08472123499579763),
 'phase_diff': np.float64(0.0009466481397022505)}

In [299]:
# Generate a DataFrame with perturbed values for all rows in fft_test
num_rows_test = len(dataframe_testdata)
perturbed_test_df = add_perturbation(reference_features_test, num_rows_test, perturb_range=0.03)
perturbed_test_df

,mean_fft,std_fft,max_fft,min_fft,median_fft,sum_fft,spectral_entropy,spectral_centroid,spectral_bandwidth,phase_variance,phase_mean,phase_diff
0,308.836035,466.578572,17795.796832,7.118809,276.904248,644515.268326,7.169227,498687.451485,291268.738664,3.246888,-0.084805,0.000951
1,311.346540,476.696153,17415.457566,7.478032,275.958219,645260.618452,7.366490,501581.429406,297092.412561,3.383871,-0.087093,0.000956
2,316.514555,459.347573,17839.986840,7.180373,277.428180,631145.927256,7.493206,484248.081910,304218.208695,3.268926,-0.084031,0.000953
3,315.458520,471.001421,17335.697525,7.512186,273.928481,650051.432826,7.339546,488926.599964,303630.246240,3.310509,-0.086533,0.000936
4,308.919621,453.055013,17119.228802,7.206011,271.599243,636327.892578,7.466674,481684.619762,300906.184348,3.325453,-0.086293,0.000970
...,...,...,...,...,...,...,...,...,...,...,...,...
1995,324.932855,471.077804,17725.652415,7.517432,267.806019,634327.777638,7.435797,495971.831979,292686.399498,3.319928,-0.085742,0.000961
1996,320.988721,453.424097,17172.446752,7.195809,283.190394,613386.162875,7.218126,490223.082877,297606.046187,3.307333,-0.082422,0.000941
1997,312.423804,469.068443,17629.111688,7.091874,280.565845,647677.652177,7.311947,504036.848154,306209.018940,3.288837,-0.086651,0.000966
1998,317.644647,459.520768,17365.256197,7.196446,268.196721,614668.404695,7.251970,493506.479515,306413.986097,3.235014,-0.082596,0.000928


In [300]:
dataframe_testdata

,Frequency,FFT Magnitude,Phase,Object_Presence
0,0.00000,17591.160470,0.000000,0.0
1,488.28125,8624.422770,-3.116442,0.0
2,976.56250,269.932200,2.781762,0.0
3,1464.84375,101.451438,1.259705,0.0
4,1953.12500,159.854316,-0.154286,0.0
...,...,...,...,...
1995,974121.09375,382.129825,0.512916,0.0
1996,974609.37500,385.904814,-2.873235,0.0
1997,975097.65625,363.592039,-0.043251,0.0
1998,975585.93750,330.245526,-2.947535,0.0


In [301]:
len(perturbed_test_df),len(dataframe_testdata)

(2000, 2000)

In [302]:
# Ensure both DataFrames have the same number of rows
if len(dataframe_testdata) != len(perturbed_test_df):
    print("The DataFrames have different lengths. Adjust them to have the same number of rows before concatenating.")
else:
    # Concatenate the DataFrames side by side
    dataframe_testdata_withfeatures = pd.concat([dataframe_testdata, perturbed_test_df], axis=1)

# Display the first few rows of the updated dataframe_testdata to verify
dataframe_testdata_withfeatures.head()

,Frequency,FFT Magnitude,Phase,Object_Presence,mean_fft,std_fft,max_fft,min_fft,median_fft,sum_fft,spectral_entropy,spectral_centroid,spectral_bandwidth,phase_variance,phase_mean,phase_diff
0,0.00000,17591.160470,0.000000,0.0,308.836035,466.578572,17795.796832,7.118809,276.904248,644515.268326,7.169227,498687.451485,291268.738664,3.246888,-0.084805,0.000951
1,488.28125,8624.422770,-3.116442,0.0,311.346540,476.696153,17415.457566,7.478032,275.958219,645260.618452,7.366490,501581.429406,297092.412561,3.383871,-0.087093,0.000956
2,976.56250,269.932200,2.781762,0.0,316.514555,459.347573,17839.986840,7.180373,277.428180,631145.927256,7.493206,484248.081910,304218.208695,3.268926,-0.084031,0.000953
3,1464.84375,101.451438,1.259705,0.0,315.458520,471.001421,17335.697525,7.512186,273.928481,650051.432826,7.339546,488926.599964,303630.246240,3.310509,-0.086533,0.000936
4,1953.12500,159.854316,-0.154286,0.0,308.919621,453.055013,17119.228802,7.206011,271.599243,636327.892578,7.466674,481684.619762,300906.184348,3.325453,-0.086293,0.000970


In [303]:
len(dataframe_testdata_withfeatures)

2000

In [304]:
dataframe_testdata_withfeatures

,Frequency,FFT Magnitude,Phase,Object_Presence,mean_fft,std_fft,max_fft,min_fft,median_fft,sum_fft,spectral_entropy,spectral_centroid,spectral_bandwidth,phase_variance,phase_mean,phase_diff
0,0.00000,17591.160470,0.000000,0.0,308.836035,466.578572,17795.796832,7.118809,276.904248,644515.268326,7.169227,498687.451485,291268.738664,3.246888,-0.084805,0.000951
1,488.28125,8624.422770,-3.116442,0.0,311.346540,476.696153,17415.457566,7.478032,275.958219,645260.618452,7.366490,501581.429406,297092.412561,3.383871,-0.087093,0.000956
2,976.56250,269.932200,2.781762,0.0,316.514555,459.347573,17839.986840,7.180373,277.428180,631145.927256,7.493206,484248.081910,304218.208695,3.268926,-0.084031,0.000953
3,1464.84375,101.451438,1.259705,0.0,315.458520,471.001421,17335.697525,7.512186,273.928481,650051.432826,7.339546,488926.599964,303630.246240,3.310509,-0.086533,0.000936
4,1953.12500,159.854316,-0.154286,0.0,308.919621,453.055013,17119.228802,7.206011,271.599243,636327.892578,7.466674,481684.619762,300906.184348,3.325453,-0.086293,0.000970
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,974121.09375,382.129825,0.512916,0.0,324.932855,471.077804,17725.652415,7.517432,267.806019,634327.777638,7.435797,495971.831979,292686.399498,3.319928,-0.085742,0.000961
1996,974609.37500,385.904814,-2.873235,0.0,320.988721,453.424097,17172.446752,7.195809,283.190394,613386.162875,7.218126,490223.082877,297606.046187,3.307333,-0.082422,0.000941
1997,975097.65625,363.592039,-0.043251,0.0,312.423804,469.068443,17629.111688,7.091874,280.565845,647677.652177,7.311947,504036.848154,306209.018940,3.288837,-0.086651,0.000966
1998,975585.93750,330.245526,-2.947535,0.0,317.644647,459.520768,17365.256197,7.196446,268.196721,614668.404695,7.251970,493506.479515,306413.986097,3.235014,-0.082596,0.000928


In [305]:
# You can directly test whether the model is predicting this set of data as true or not by running it through loaded model 
# You can skip the second half of the dateset and start from the dropping features section

# Carrierseat data

In [306]:
import os
import numpy as np
import pandas as pd

# Define the dataset file name
filename = "test_Carrierseat_withoutbaby_npy_array_Lowpassfiltered_withlabel_1.npy"

# Build full path (2 levels up -> Data/Processed/)
file_path = os.path.join("..", "..", "Data", "Test", filename)

# Load the NumPy array
loaded_array = np.load(file_path, mmap_mode='r')

# Convert to DataFrame
dataframe_carrierseat = pd.DataFrame(
    loaded_array,
    columns=['Frequency', 'FFT Magnitude', 'Phase', 'Object_Presence']
)

print(f"Data loaded from: {os.path.abspath(file_path)}")
dataframe_carrierseat


Data loaded from: /Users/mandarkale/Documents/MyProjects/MachineLearning/FIUS_Based_Infant_Presence_Detection_in_Car_Seats/source_code/Data/Test/test_Carrierseat_withoutbaby_npy_array_Lowpassfiltered_withlabel_1.npy


,Frequency,FFT Magnitude,Phase,Object_Presence
0,0.00000,16732.464039,0.000000,1.0
1,488.28125,8333.605309,3.112956,1.0
2,976.56250,204.398557,-1.629384,1.0
3,1464.84375,124.749508,1.244306,1.0
4,1953.12500,218.879548,-2.558848,1.0
...,...,...,...,...
3995,-2441.40625,112.613316,-0.375541,1.0
3996,-1953.12500,218.879548,2.558848,1.0
3997,-1464.84375,124.749508,-1.244306,1.0
3998,-976.56250,204.398557,1.629384,1.0


In [307]:
fft_magnitudes_carrier = dataframe_carrierseat["FFT Magnitude"].values  # Convert to NumPy array
fft_frequencies_carrier = dataframe_carrierseat["Frequency"].values  # Convert to NumPy array
fft_phase_carrier = dataframe_carrierseat["Phase"].values  # Convert to NumPy array


In [308]:
# Extract features for fft_carrier
reference_features_carrier = extract_features(fft_frequencies_carrier, fft_magnitudes_carrier, fft_phase_carrier)
reference_features_carrier

{'mean_fft': np.float64(325.95225423295017),
 'std_fft': np.float64(378.9192634886786),
 'max_fft': np.float64(16732.464038774917),
 'min_fft': np.float64(8.289574826003792),
 'median_fft': np.float64(275.93543287593525),
 'sum_fft': np.float64(1303809.0169318006),
 'spectral_entropy': np.float64(8.0351420470818),
 'spectral_centroid': np.float64(-70.97002611338394),
 'spectral_bandwidth': np.float64(579371.9519415188),
 'phase_variance': np.float64(3.2899656084123654),
 'phase_mean': np.float64(5.3290705182007515e-18),
 'phase_diff': np.float64(-0.0007784337064980223)}

In [309]:
# Generate a DataFrame with perturbed values for all rows in fft_empty
num_rows_carrier = len(dataframe_carrierseat)
perturbed_carrier_df = add_perturbation(reference_features_carrier, num_rows_carrier, perturb_range=0.03)
perturbed_carrier_df

,mean_fft,std_fft,max_fft,min_fft,median_fft,sum_fft,spectral_entropy,spectral_centroid,spectral_bandwidth,phase_variance,phase_mean,phase_diff
0,329.623856,375.028735,16344.601676,8.363746,273.198169,1.322001e+06,8.197688,-70.096109,585430.289798,3.374365,5.337643e-18,-0.000799
1,322.883004,373.885104,16456.883210,8.121512,281.601845,1.323844e+06,7.836100,-72.257700,563204.414094,3.198848,5.185762e-18,-0.000796
2,319.254691,372.006769,16738.904541,8.174651,283.156261,1.274858e+06,8.195555,-71.871172,565516.395528,3.326826,5.331693e-18,-0.000781
3,325.573751,381.641313,17206.782314,8.171157,272.179797,1.318709e+06,7.855679,-72.264670,578767.369845,3.296219,5.480205e-18,-0.000790
4,328.892359,370.363153,16597.980866,8.043605,281.626472,1.285396e+06,7.991595,-72.970766,588348.771928,3.267600,5.180792e-18,-0.000780
...,...,...,...,...,...,...,...,...,...,...,...,...
3995,322.174718,387.577755,16342.025950,8.124997,279.383871,1.294459e+06,7.969928,-69.572279,588288.609508,3.226339,5.258754e-18,-0.000788
3996,325.374487,382.086220,16601.266412,8.457349,274.483040,1.280128e+06,7.967541,-72.628450,570269.269419,3.233621,5.284439e-18,-0.000769
3997,330.845753,386.410450,17048.429257,8.469112,278.127676,1.302813e+06,8.130289,-70.350359,580871.048837,3.224408,5.298037e-18,-0.000798
3998,330.114177,384.332329,16479.246151,8.414339,275.725166,1.268892e+06,8.160208,-69.984386,576982.947626,3.379807,5.334591e-18,-0.000764


In [310]:
dataframe_carrierseat

,Frequency,FFT Magnitude,Phase,Object_Presence
0,0.00000,16732.464039,0.000000,1.0
1,488.28125,8333.605309,3.112956,1.0
2,976.56250,204.398557,-1.629384,1.0
3,1464.84375,124.749508,1.244306,1.0
4,1953.12500,218.879548,-2.558848,1.0
...,...,...,...,...
3995,-2441.40625,112.613316,-0.375541,1.0
3996,-1953.12500,218.879548,2.558848,1.0
3997,-1464.84375,124.749508,-1.244306,1.0
3998,-976.56250,204.398557,1.629384,1.0


In [311]:
len(perturbed_carrier_df),len(dataframe_carrierseat)

(4000, 4000)

In [312]:

# Ensure both DataFrames have the same number of rows
if len(dataframe_carrierseat) != len(perturbed_carrier_df):
    print("The DataFrames have different lengths. Adjust them to have the same number of rows before concatenating.")
else:
    # Concatenate the DataFrames side by side
    dataframe_carrierseat_withfeatures = pd.concat([dataframe_carrierseat, perturbed_carrier_df], axis=1)

# Display the first few rows of the updated dataframe_emptyseat to verify
dataframe_carrierseat_withfeatures.head()

,Frequency,FFT Magnitude,Phase,Object_Presence,mean_fft,std_fft,max_fft,min_fft,median_fft,sum_fft,spectral_entropy,spectral_centroid,spectral_bandwidth,phase_variance,phase_mean,phase_diff
0,0.00000,16732.464039,0.000000,1.0,329.623856,375.028735,16344.601676,8.363746,273.198169,1.322001e+06,8.197688,-70.096109,585430.289798,3.374365,5.337643e-18,-0.000799
1,488.28125,8333.605309,3.112956,1.0,322.883004,373.885104,16456.883210,8.121512,281.601845,1.323844e+06,7.836100,-72.257700,563204.414094,3.198848,5.185762e-18,-0.000796
2,976.56250,204.398557,-1.629384,1.0,319.254691,372.006769,16738.904541,8.174651,283.156261,1.274858e+06,8.195555,-71.871172,565516.395528,3.326826,5.331693e-18,-0.000781
3,1464.84375,124.749508,1.244306,1.0,325.573751,381.641313,17206.782314,8.171157,272.179797,1.318709e+06,7.855679,-72.264670,578767.369845,3.296219,5.480205e-18,-0.000790
4,1953.12500,218.879548,-2.558848,1.0,328.892359,370.363153,16597.980866,8.043605,281.626472,1.285396e+06,7.991595,-72.970766,588348.771928,3.267600,5.180792e-18,-0.000780


In [313]:
# Vertical concatenation
dataset_1_rawdata = pd.concat([dataframe_testdata,dataframe_carrierseat])
dataset_1_rawdata

,Frequency,FFT Magnitude,Phase,Object_Presence
0,0.00000,17591.160470,0.000000,0.0
1,488.28125,8624.422770,-3.116442,0.0
2,976.56250,269.932200,2.781762,0.0
3,1464.84375,101.451438,1.259705,0.0
4,1953.12500,159.854316,-0.154286,0.0
...,...,...,...,...
3995,-2441.40625,112.613316,-0.375541,1.0
3996,-1953.12500,218.879548,2.558848,1.0
3997,-1464.84375,124.749508,-1.244306,1.0
3998,-976.56250,204.398557,1.629384,1.0


In [314]:
# Resetting the index if unique indices are needed
dataset_1_rawdata = dataset_1_rawdata.reset_index(drop=True)
dataset_1_rawdata

,Frequency,FFT Magnitude,Phase,Object_Presence
0,0.00000,17591.160470,0.000000,0.0
1,488.28125,8624.422770,-3.116442,0.0
2,976.56250,269.932200,2.781762,0.0
3,1464.84375,101.451438,1.259705,0.0
4,1953.12500,159.854316,-0.154286,0.0
...,...,...,...,...
5995,-2441.40625,112.613316,-0.375541,1.0
5996,-1953.12500,218.879548,2.558848,1.0
5997,-1464.84375,124.749508,-1.244306,1.0
5998,-976.56250,204.398557,1.629384,1.0


In [315]:
dataset_1_withfeatures = pd.concat([dataframe_testdata_withfeatures,dataframe_carrierseat_withfeatures])
dataset_1_withfeatures

,Frequency,FFT Magnitude,Phase,Object_Presence,mean_fft,std_fft,max_fft,min_fft,median_fft,sum_fft,spectral_entropy,spectral_centroid,spectral_bandwidth,phase_variance,phase_mean,phase_diff
0,0.00000,17591.160470,0.000000,0.0,308.836035,466.578572,17795.796832,7.118809,276.904248,6.445153e+05,7.169227,498687.451485,291268.738664,3.246888,-8.480529e-02,0.000951
1,488.28125,8624.422770,-3.116442,0.0,311.346540,476.696153,17415.457566,7.478032,275.958219,6.452606e+05,7.366490,501581.429406,297092.412561,3.383871,-8.709325e-02,0.000956
2,976.56250,269.932200,2.781762,0.0,316.514555,459.347573,17839.986840,7.180373,277.428180,6.311459e+05,7.493206,484248.081910,304218.208695,3.268926,-8.403063e-02,0.000953
3,1464.84375,101.451438,1.259705,0.0,315.458520,471.001421,17335.697525,7.512186,273.928481,6.500514e+05,7.339546,488926.599964,303630.246240,3.310509,-8.653307e-02,0.000936
4,1953.12500,159.854316,-0.154286,0.0,308.919621,453.055013,17119.228802,7.206011,271.599243,6.363279e+05,7.466674,481684.619762,300906.184348,3.325453,-8.629271e-02,0.000970
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,-2441.40625,112.613316,-0.375541,1.0,322.174718,387.577755,16342.025950,8.124997,279.383871,1.294459e+06,7.969928,-69.572279,588288.609508,3.226339,5.258754e-18,-0.000788
3996,-1953.12500,218.879548,2.558848,1.0,325.374487,382.086220,16601.266412,8.457349,274.483040,1.280128e+06,7.967541,-72.628450,570269.269419,3.233621,5.284439e-18,-0.000769
3997,-1464.84375,124.749508,-1.244306,1.0,330.845753,386.410450,17048.429257,8.469112,278.127676,1.302813e+06,8.130289,-70.350359,580871.048837,3.224408,5.298037e-18,-0.000798
3998,-976.56250,204.398557,1.629384,1.0,330.114177,384.332329,16479.246151,8.414339,275.725166,1.268892e+06,8.160208,-69.984386,576982.947626,3.379807,5.334591e-18,-0.000764


In [316]:
dataset_1_withfeatures = dataset_1_withfeatures.reset_index(drop=True)

In [317]:
dataframe_testdata_withfeatures = dataset_1_withfeatures

In [318]:
dataframe_testdata_withfeatures

,Frequency,FFT Magnitude,Phase,Object_Presence,mean_fft,std_fft,max_fft,min_fft,median_fft,sum_fft,spectral_entropy,spectral_centroid,spectral_bandwidth,phase_variance,phase_mean,phase_diff
0,0.00000,17591.160470,0.000000,0.0,308.836035,466.578572,17795.796832,7.118809,276.904248,6.445153e+05,7.169227,498687.451485,291268.738664,3.246888,-8.480529e-02,0.000951
1,488.28125,8624.422770,-3.116442,0.0,311.346540,476.696153,17415.457566,7.478032,275.958219,6.452606e+05,7.366490,501581.429406,297092.412561,3.383871,-8.709325e-02,0.000956
2,976.56250,269.932200,2.781762,0.0,316.514555,459.347573,17839.986840,7.180373,277.428180,6.311459e+05,7.493206,484248.081910,304218.208695,3.268926,-8.403063e-02,0.000953
3,1464.84375,101.451438,1.259705,0.0,315.458520,471.001421,17335.697525,7.512186,273.928481,6.500514e+05,7.339546,488926.599964,303630.246240,3.310509,-8.653307e-02,0.000936
4,1953.12500,159.854316,-0.154286,0.0,308.919621,453.055013,17119.228802,7.206011,271.599243,6.363279e+05,7.466674,481684.619762,300906.184348,3.325453,-8.629271e-02,0.000970
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,-2441.40625,112.613316,-0.375541,1.0,322.174718,387.577755,16342.025950,8.124997,279.383871,1.294459e+06,7.969928,-69.572279,588288.609508,3.226339,5.258754e-18,-0.000788
5996,-1953.12500,218.879548,2.558848,1.0,325.374487,382.086220,16601.266412,8.457349,274.483040,1.280128e+06,7.967541,-72.628450,570269.269419,3.233621,5.284439e-18,-0.000769
5997,-1464.84375,124.749508,-1.244306,1.0,330.845753,386.410450,17048.429257,8.469112,278.127676,1.302813e+06,8.130289,-70.350359,580871.048837,3.224408,5.298037e-18,-0.000798
5998,-976.56250,204.398557,1.629384,1.0,330.114177,384.332329,16479.246151,8.414339,275.725166,1.268892e+06,8.160208,-69.984386,576982.947626,3.379807,5.334591e-18,-0.000764


# dropping features

In [319]:
# Resetting the index if unique indices are needed
dataframe_testdata_withfeatures = dataframe_testdata_withfeatures.reset_index(drop=True)
dataframe_testdata_withfeatures

,Frequency,FFT Magnitude,Phase,Object_Presence,mean_fft,std_fft,max_fft,min_fft,median_fft,sum_fft,spectral_entropy,spectral_centroid,spectral_bandwidth,phase_variance,phase_mean,phase_diff
0,0.00000,17591.160470,0.000000,0.0,308.836035,466.578572,17795.796832,7.118809,276.904248,6.445153e+05,7.169227,498687.451485,291268.738664,3.246888,-8.480529e-02,0.000951
1,488.28125,8624.422770,-3.116442,0.0,311.346540,476.696153,17415.457566,7.478032,275.958219,6.452606e+05,7.366490,501581.429406,297092.412561,3.383871,-8.709325e-02,0.000956
2,976.56250,269.932200,2.781762,0.0,316.514555,459.347573,17839.986840,7.180373,277.428180,6.311459e+05,7.493206,484248.081910,304218.208695,3.268926,-8.403063e-02,0.000953
3,1464.84375,101.451438,1.259705,0.0,315.458520,471.001421,17335.697525,7.512186,273.928481,6.500514e+05,7.339546,488926.599964,303630.246240,3.310509,-8.653307e-02,0.000936
4,1953.12500,159.854316,-0.154286,0.0,308.919621,453.055013,17119.228802,7.206011,271.599243,6.363279e+05,7.466674,481684.619762,300906.184348,3.325453,-8.629271e-02,0.000970
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,-2441.40625,112.613316,-0.375541,1.0,322.174718,387.577755,16342.025950,8.124997,279.383871,1.294459e+06,7.969928,-69.572279,588288.609508,3.226339,5.258754e-18,-0.000788
5996,-1953.12500,218.879548,2.558848,1.0,325.374487,382.086220,16601.266412,8.457349,274.483040,1.280128e+06,7.967541,-72.628450,570269.269419,3.233621,5.284439e-18,-0.000769
5997,-1464.84375,124.749508,-1.244306,1.0,330.845753,386.410450,17048.429257,8.469112,278.127676,1.302813e+06,8.130289,-70.350359,580871.048837,3.224408,5.298037e-18,-0.000798
5998,-976.56250,204.398557,1.629384,1.0,330.114177,384.332329,16479.246151,8.414339,275.725166,1.268892e+06,8.160208,-69.984386,576982.947626,3.379807,5.334591e-18,-0.000764


In [320]:
to_drop = {'median_fft', 'sum_fft', 'min_fft', 'mean_fft', 'spectral_centroid', 'phase_mean'}
test_X = dataframe_testdata_withfeatures.drop(columns=['Object_Presence'])  # Features
test_y = dataframe_testdata_withfeatures['Object_Presence']  # Labels
test_x_dataset_1_reduced = test_X.drop(columns=to_drop)

In [321]:
test_x_dataset_1_reduced

,Frequency,FFT Magnitude,Phase,std_fft,max_fft,spectral_entropy,spectral_bandwidth,phase_variance,phase_diff
0,0.00000,17591.160470,0.000000,466.578572,17795.796832,7.169227,291268.738664,3.246888,0.000951
1,488.28125,8624.422770,-3.116442,476.696153,17415.457566,7.366490,297092.412561,3.383871,0.000956
2,976.56250,269.932200,2.781762,459.347573,17839.986840,7.493206,304218.208695,3.268926,0.000953
3,1464.84375,101.451438,1.259705,471.001421,17335.697525,7.339546,303630.246240,3.310509,0.000936
4,1953.12500,159.854316,-0.154286,453.055013,17119.228802,7.466674,300906.184348,3.325453,0.000970
...,...,...,...,...,...,...,...,...,...
5995,-2441.40625,112.613316,-0.375541,387.577755,16342.025950,7.969928,588288.609508,3.226339,-0.000788
5996,-1953.12500,218.879548,2.558848,382.086220,16601.266412,7.967541,570269.269419,3.233621,-0.000769
5997,-1464.84375,124.749508,-1.244306,386.410450,17048.429257,8.130289,580871.048837,3.224408,-0.000798
5998,-976.56250,204.398557,1.629384,384.332329,16479.246151,8.160208,576982.947626,3.379807,-0.000764


In [322]:
test_y

0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
       ... 
5995    1.0
5996    1.0
5997    1.0
5998    1.0
5999    1.0
Name: Object_Presence, Length: 6000, dtype: float64

# Testing the Xg Boost Model

In [323]:
import os
import joblib
import xgboost as xgb

# Define directories
misc_dir = os.path.join("..", "..", "Misc")
model_dir = os.path.join("..", "..", "Models", "Baby Detection and Seat Classification XGBoost")

# Define file paths
model_path = os.path.join(misc_dir, "xgboost_model_Seat_classification.json")
scaler_path = os.path.join(model_dir, "xgboost_scaler_Seat_classification.pkl")

# Load the trained XGBoost model
clf = xgb.XGBClassifier()
clf.load_model(model_path)

# Load the scaler
scaler = joblib.load(scaler_path)

print(f"Model loaded from: {os.path.abspath(model_path)}")
print(f"Scaler loaded from: {os.path.abspath(scaler_path)}")

# Transform test data
X_test_scaled = scaler.transform(test_x_dataset_1_reduced)

# Predict
y_pred = clf.predict(X_test_scaled)
y_pred

Model loaded from: /Users/mandarkale/Documents/MyProjects/MachineLearning/FIUS_Based_Infant_Presence_Detection_in_Car_Seats/source_code/Misc/xgboost_model_Seat_classification.json
Scaler loaded from: /Users/mandarkale/Documents/MyProjects/MachineLearning/FIUS_Based_Infant_Presence_Detection_in_Car_Seats/source_code/Models/Baby Detection and Seat Classification XGBoost/xgboost_scaler_Seat_classification.pkl


ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- max_fft
- phase_diff
- spectral_bandwidth
- std_fft


In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Accuracy
accuracy = accuracy_score(test_y, y_pred)
print(f"Accuracy: {accuracy:.4f}")

# Optional: More detailed report
print("\nClassification Report:")
print(classification_report(test_y, y_pred))

# Optional: Confusion matrix
print("\nConfusion Matrix:")
print(confusion_matrix(test_y, y_pred))


In [ ]:
# Generate confusion matrix
cm = confusion_matrix(test_y, y_pred)
cm

fig, ax = plt.subplots(figsize=(6, 6))
sns.heatmap(cm, annot=True, fmt='d', ax=ax)  # Use fmt='d' for integer annotation
plt.xlabel("Predicted label")
plt.ylabel("True label")
plt.title('Confusion Matrix for test dataset')
plt.show()

###### 

# Testing with MLP model task 1

In [ ]:
import os
from tensorflow.keras.models import load_model
import joblib

# Define directories
model_dir = os.path.join("..", "..", "Models", "Seat Classification")

# Define file paths
model_path = os.path.join(model_dir, "mlp_model_Seat_Classification.h5")
scaler_path = os.path.join(model_dir, "mlp_scaler_Seat_Classification.pkl")

# Load the trained MLP model
mlp_model = load_model(model_path)

# Load the scaler
scaler = joblib.load(scaler_path)

print(f"MLP model loaded from: {os.path.abspath(model_path)}")
print(f"Scaler loaded from: {os.path.abspath(scaler_path)}")

# Transform new test data
X_test_scaled = scaler.transform(test_x_dataset_1_reduced)

# Predict probabilities
y_pred_prob = mlp_model.predict(X_test_scaled)

# Convert to binary classification (0 or 1)
y_pred = (y_pred_prob > 0.5).astype(int).flatten()

y_pred

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Accuracy
accuracy = accuracy_score(test_y, y_pred)
print(f"Accuracy: {accuracy:.4f}")

# Optional: More detailed report
print("\nClassification Report:")
print(classification_report(test_y, y_pred))

# Optional: Confusion matrix
print("\nConfusion Matrix:")
print(confusion_matrix(test_y, y_pred))


In [ ]:
# Generate confusion matrix
cm = confusion_matrix(test_y, y_pred)
cm

fig, ax = plt.subplots(figsize=(6, 6))
sns.heatmap(cm, annot=True, fmt='d', ax=ax)  # Use fmt='d' for integer annotation
plt.xlabel("Predicted label")
plt.ylabel("True label")
plt.title('Confusion Matrix for test dataset')
plt.show()